In [3]:
import requests
from groq import Groq


In [13]:
# Tool 1: Currency Exchange Rate
def get_exchange_rate(base_currency, target_currency):
    response = requests.get(f"https://api.exchangerate-api.com/v4/latest/{base_currency}")
    data = response.json()

    if 'rates' in data and target_currency in data['rates']:
        return data['rates'][target_currency]
    else:
        return f"Exchange rate not found for {base_currency} to {target_currency}"

# Tool 2: Closest Currency in Value
def get_closest_currency(base_currency):
    response = requests.get(f"https://api.exchangerate-api.com/v4/latest/{base_currency}")
    data = response.json()

    if 'rates' in data:
        base_rate = data['rates'][base_currency]  # The base currency's own rate (should be 1)
        closest_currency = None
        closest_difference = float('inf')

        # Loop through all available currencies and find the closest one in value
        for currency, rate in data['rates'].items():
            if currency != base_currency:  # Ignore the base currency itself
                difference = abs(1 - rate)  # Calculate the difference between 1 (the base rate) and the rate of the currency
                if difference < closest_difference:
                    closest_difference = difference
                    closest_currency = currency

        return f"The closest currency to {base_currency} is {closest_currency}."
    else:
        return "No data available for closest currency."

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_exchange_rate",
            "description": "Get exchange rate between two currencies.",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_currency": {"type": "string", "description": "The currency to convert from (e.g., USD)."},
                    "target_currency": {"type": "string", "description": "The currency to convert to (e.g., EUR)."}
                },
                "required": ["base_currency", "target_currency"],
                "additionalProperties": False
            },
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_closest_currency",
            "description": "Get the currency with the closest exchange rate to the base currency.",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_currency": {"type": "string", "description": "The base currency to compare against (e.g., USD)."}
                },
                "required": ["base_currency"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

client = Groq(api_key='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')

messages = [{"role": "user", "content": "What is the exchange rate from USD to EUR?"}]

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=messages,
    tools=tools
)


In [14]:
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_f5rb', function=Function(arguments='{"base_currency":"USD","target_currency":"EUR"}', name='get_exchange_rate'), type='function')])


In [15]:
import rich
rich.print(response.choices[0].message)

ChatCompletionMessage(
    content=None,
    role='assistant',
    function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_f5rb',
            function=Function(
                arguments='{"base_currency":"USD","target_currency":"EUR"}',
                name='get_exchange_rate'
            ),
            type='function'
        )
    ]
)

In [16]:
import json
tool_call = response.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

print(args)

result = get_closest_currency(args["base_currency"])
result

{'base_currency': 'USD', 'target_currency': 'EUR'}


'The closest currency to USD is BMD.'

In [17]:
result = get_exchange_rate(args["base_currency"], args["target_currency"])
result

0.972